# <u>Training Exercices - Endogeneity: Testing<u>

__<u>Jorge Pineño Pérez<u>__ / Module 5 - Endogeneity

In [1]:
# package imports
install.packages("readxl")
install.packages("dplyr")
library(readxl)
library(dplyr)

Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'readxl' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\Rtmp8CqSuA\downloaded_packages


Installing package into 'C:/Users/USER/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\USER\AppData\Local\Temp\Rtmp8CqSuA\downloaded_packages



Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




__<u>Training Exercise 4.4<u>__

In [30]:
# import the gasoline market dataframe
gas <- read_excel('TrainExer44.xls')
head(gas)

OBS,GC,PG,RI,RPN,RPT,RPU
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1970,6.714227,-0.10094651,2.943489,0.3118717,-0.09737416,-0.2180022
1971,6.765328,-0.13722132,2.985432,0.3096610,-0.06899287,-0.2047944
1972,6.830206,-0.15666032,3.033986,0.2689674,-0.06167182,-0.2333631
1973,6.847757,-0.12389238,3.093292,0.2104507,-0.11188828,-0.2321934
1974,6.697846,0.07443991,3.079637,0.1607933,-0.19415601,-0.2951473
1975,6.732342,0.05303518,3.089182,0.1562727,-0.21251253,-0.2056396


__(b)__

In [31]:
# regress PG (X) on a constant, RI, RPT, RPN, and RPU (Z), to get the explained part
PG_model <- lm(PG ~ RI + RPT + RPN + RPU, data=gas)
summary(PG_model)


Call:
lm(formula = PG ~ RI + RPT + RPN + RPU, data = gas)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.160161 -0.036024 -0.002891  0.034824  0.175059 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   7.7410     0.8337   9.285 1.40e-09 ***
RI           -2.2984     0.2471  -9.303 1.35e-09 ***
RPT          -0.8080     0.1912  -4.225 0.000277 ***
RPN          -3.5279     0.3520 -10.023 3.06e-10 ***
RPU           0.2331     0.1831   1.273 0.214765    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.07269 on 25 degrees of freedom
Multiple R-squared:  0.8868,	Adjusted R-squared:  0.8687 
F-statistic: 48.97 on 4 and 25 DF,  p-value: 1.797e-11


In [32]:
# add PG_fitted to the tibble
gas$PG_fitted <- fitted(PG_model)

# regress GC on PG_fitted and RI, which will give the estimate of beta_2 as the coefficient of PG_fitted
GC_model <- lm(GC ~ PG_fitted + RI, data=gas)
summary(GC_model)


Call:
lm(formula = GC ~ PG_fitted + RI, data = gas)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.054154 -0.025003 -0.008033  0.012243  0.090373 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  5.01370    0.13386   37.46  < 2e-16 ***
PG_fitted   -0.54445    0.04618  -11.79 3.72e-12 ***
RI           0.56466    0.04050   13.94 7.46e-14 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.03833 on 27 degrees of freedom
Multiple R-squared:  0.9668,	Adjusted R-squared:  0.9644 
F-statistic: 393.3 on 2 and 27 DF,  p-value: < 2.2e-16


__(c)__

In [33]:
# calculate the errors of the 2SLS
e2SLS <- resid(GC_model)

# regress the e2SLS on the instruments (Z) RI, RPT, RPN, and RPU
e2SLS_model <- lm(e2SLS ~ RI + RPT + RPN + RPU, data=gas)
summary(e2SLS_model)


Call:
lm(formula = e2SLS ~ RI + RPT + RPN + RPU, data = gas)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.067822 -0.020037 -0.005752  0.022093  0.084996 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept) -0.20975    0.44738  -0.469    0.643
RI           0.06041    0.13258   0.456    0.653
RPT         -0.05120    0.10261  -0.499    0.622
RPN          0.02041    0.18887   0.108    0.915
RPU         -0.07023    0.09826  -0.715    0.481

Residual standard error: 0.03901 on 25 degrees of freedom
Multiple R-squared:  0.04093,	Adjusted R-squared:  -0.1125 
F-statistic: 0.2667 on 4 and 25 DF,  p-value: 0.8965


In [34]:
# compute the Sargan test statistic as n*R-squared,
# knowing that the cirtical value (5%) chi-squared distribution
# with (5-3)df is 5.99, and with anything below that, the null hypothesis
# is not rejected, and the instruments seem to therefore be valid
sargan <- nobs(e2SLS_model) * summary(e2SLS_model)$r.squared
sargan

[1] 1.227949